# **The Wisdom of the Crowd in LLMs**
### **Abstract** 
We analyze the performance of a group of LLMs as compared to the performance of its constituent individuals.  
We do this by performing a majority vote on the numbers that appear in the final answer.  
In this notebook, we use the DRAW-1K and ALG-514 datasets.  

This method only the numbers that appear the majority of the time. (majority = greater than half the time. ) 
However, this method lacks real-world use-case especially in cases where an explanation is needed.  

Later, we propose a method whereby we instead elect a response where the produced numbers have the lowest levenshtein distance and compare the results.  
This method will provide users with an explanation and is much more useful for real-world use cases.  

**Potential caveats in this experiment:**  
- **We extract all decimals and fractions from ChatGPT's response.** It is possible that ChatGPT simply mentioned the answer in its response.  
However, based on a short preliminary check that I have made of the dataset, it appears that, most of the time, when ChatGPT mentions the correct answer in its response, it usually is correct. Furthermore, we take steps to avoid this.  
- **The majority solution might be larger than any individual ChatGPT response** There certainly might be places where this is the case. However, we show that, in most cases, the majority solution is smaller than any constituent individual's response.

## **1. Data preparation**
### **Download libraries**

In [1]:
%%capture
%pip install pandas==1.3.5
%pip install scipy==1.7.3

### **Load libraries**
In order to replicate the results of the experiment, I have included the Python versions as well as the versions of the libraries used as a comment.

In [2]:
# PYTHON VERSION ---------------- #
# - Python v3.7.8                 #
#                                 #
# LIBRARIES --------------------- #
# - pandas v1.3.5                 #
# - scipy  v1.7.3                 #
# =============================== #

import re

# ------------------------------- #
# Pandas
# ------------------------------- #
import pandas

# ------------------------------- #
# Scipy
# ------------------------------- #
from scipy.spatial.distance import pdist

### **Constants**
Various constants are specified. These are changed from run-to-run.  

In [3]:
DATASET = 'draw'
N_JOBS = 10

### **File-load utility functions**
We provide various utility functions for loading files.  

In [4]:
def load_file(num : int) -> pandas.DataFrame:
    file_path = f'data/{DATASET}/sample_{num}.jsonl'
    data = pandas.read_json(file_path,lines=True)
    data = data[['question_number', 'response']]
    data = data.set_index('question_number')
    data = data.rename(columns={'response': f'sample_{num}'})
    return data

def load_n_jobs_file() -> pandas.DataFrame:
    dataframes = load_file(0)
    for i in range(1, N_JOBS):
        dataframes = dataframes.join(load_file(i))
    return dataframes

### **Load files**

In [5]:
# ground is the question dataset which we will use to compare our solutions to.
ground = pandas.read_json(f'data/{DATASET}/ground.json')
ground = ground[['lSolutions']]
ground.head(1)

,lSolutions
0,[2.14285714286]


In [6]:
# responses is the dataframe of N_JOBS responses from ChatGPT
responses = load_n_jobs_file().sort_index()
responses = responses.join(ground)
responses.head(1)

## **2. Majority**
As a first step, we create a majority array consisting of numbers that have appeared a majority number of times.

### **Extract decimals from response**

In [ ]:
def number_strip(text : str):
    while len(text) > 0 and (not (text[0].isdigit() or text[0] == '-') or text[0] == '0') and text: text = text[1:]
    while len(text) > 0 and not text[-1].isdigit(): text = text.rstrip(text[-1])
    return text

def extract_decimals(text : str):
    text = str(text)
    # We try to limit the number of lucky responses ChatGPT can get.
    # There are cases, where it names variables like so: t_1, we consider this case and do not consider the 1
    text = re.sub(r'[a-zA-Z]\d*|-?\d*[a-zA-Z]', ' ', text)

    # Then, we remove things that are not needed to make a number. For example, the only characters 
    # that could possibly constitute ChatGPT's response are assumed to be digits, '.', '/', '-'. However, there are certainly cases 
    # where ChatGPT uses LaTEX. We will consider that too.
    text = re.sub(r'[^0-9\.\/\-]', ' ', text)

    # Split it by spaces
    split_text = text.split(' ')
    
    # Remove cases where it is empty
    split_text = filter(lambda text : len(number_strip(text)) > 0, split_text)
    split_text = [number_strip(text) for text in split_text]
    
    decimals = [eval(text) for text in split_text]

    return list(set(decimals))

In [ ]:
decimals = responses.copy()

for i in range(N_JOBS):
    decimals[f'sample_{i}'] = decimals[f'sample_{i}'].apply(lambda row : extract_decimals(row))

decimals.head(1)

,sample_0,sample_1,sample_2,sample_3,sample_4,sample_5,sample_6,sample_7,sample_8,sample_9,lSolutions
question_number,,,,,,,,,,,
0,"[0.6, 1.6666666666666667, 1.6, 3, 0.625, 5, 9,...","[3.0, 5, 8, 9, 15, 24]","[0.375, 1, 1.6666666666666667, 3, 2.6666666666...","[0.625, 0.375, 3, 5, 8, 9, 15, 24]","[0.375, 1.6666666666666667, 1, 3, 2, 5, 2.6666...","[1.6666666666666667, 3, 5, 8, 9, 15, 24, 120]","[0.625, 5, 8, 9, 15, 24, 120]","[0.375, 1.6666666666666667, 2.6666666666666665...","[1.6666666666666667, 3, 5, 9, 15, 24, 120]","[1.6666666666666667, 2.6666666666666665, 3, 5,...","[15.0, 9.0]"


### **Select majority**
We select numbers that appear a majority of the time.

In [ ]:

def select_majority(row, cutoff):
    majority_map = dict()

    for i in range(N_JOBS):
        nums = row[f'sample_{i}']
        for j in nums:
            j = j
            if not j in majority_map: majority_map[j] = 1 
            else: majority_map[j] += 1
    
    ret = []
    for key, value in majority_map.items():
        if value > cutoff:
            ret.append(key)
    return ret

decimals['majority'] = decimals.apply(lambda row : select_majority(row, N_JOBS / 2), axis=1)
decimals.head(1)

,sample_0,sample_1,sample_2,sample_3,sample_4,sample_5,sample_6,sample_7,sample_8,sample_9,lSolutions,majority
question_number,,,,,,,,,,,,
0,"[0.6, 1.6666666666666667, 1.6, 3, 0.625, 5, 9,...","[3.0, 5, 8, 9, 15, 24]","[0.375, 1, 1.6666666666666667, 3, 2.6666666666...","[0.625, 0.375, 3, 5, 8, 9, 15, 24]","[0.375, 1.6666666666666667, 1, 3, 2, 5, 2.6666...","[1.6666666666666667, 3, 5, 8, 9, 15, 24, 120]","[0.625, 5, 8, 9, 15, 24, 120]","[0.375, 1.6666666666666667, 2.6666666666666665...","[1.6666666666666667, 3, 5, 9, 15, 24, 120]","[1.6666666666666667, 2.6666666666666665, 3, 5,...","[15.0, 9.0]","[1.6666666666666667, 3, 5, 9, 15, 24]"


### **Evaluate majority performance**
We evaluate the performance of the aggregate opinion as well as the opinions of the constituent individuals

In [ ]:
def check_correct(solution, answer, transform_func):
    solution = set([transform_func(s) for s in solution])
    answer = set([transform_func(s) for s in answer])

    if len(answer.intersection(solution)) == len(solution): return 'all'
    elif len(answer.intersection(solution)) > 0: return 'some'
    else: return 'none'

def base_transform_func(x): return round(x, 5)

is_correct = decimals.copy()

for i in range(N_JOBS):
    is_correct[f'sample_{i}'] = is_correct.apply(lambda row : check_correct(row['lSolutions'], row[f'sample_{i}'], base_transform_func), axis=1)

is_correct[f'majority'] = is_correct.apply(lambda row : check_correct(row['lSolutions'], row[f'majority'], base_transform_func), axis=1)

is_correct.head(1)

,sample_0,sample_1,sample_2,sample_3,sample_4,sample_5,sample_6,sample_7,sample_8,sample_9,lSolutions,majority
question_number,,,,,,,,,,,,
0,all,all,all,all,all,all,all,all,all,all,"[15.0, 9.0]",all


In [ ]:
value_cnt = pandas.DataFrame(is_correct[f'sample_0'].value_counts())
for i in range(1, N_JOBS):
    value_cnt = value_cnt.join(pandas.DataFrame(is_correct[f'sample_{i}'].value_counts()))
value_cnt = value_cnt.join(pandas.DataFrame(is_correct['majority'].value_counts()))

### **Majority summary**
It looks like the majority consistently performs better than any constituent individual.  
However, it is possible that the majority array is simply larger so of course, ChatGPT could get luckier, right?  
We show now that this is likely not the case and that this happens only in rare cases.

In [ ]:
def len_difference(row, column_name):
    majority_len = len(row[column_name])
    total = 0
    for i in range(N_JOBS):
        nums = len(row[f'sample_{i}'])
        total += majority_len - nums
    return total / N_JOBS

len_diff = pandas.DataFrame()
len_diff['len_difference'] = decimals.apply(lambda row : len_difference(row, 'majority'), axis=1)
print('NUMBER OF CASES WHERE AVERAGE INDIVIDUAL SIZE > MAJORITY SIZE:', len(len_diff[len_diff['len_difference'] > 0]))
print('AVERAGE MAJORITY - INDIVIDUAL SIZE:', len_diff['len_difference'].sum()/len(len_diff))

NUMBER OF CASES WHERE AVERAGE INDIVIDUAL SIZE > MAJORITY SIZE: 93
AVERAGE MAJORITY - INDIVIDUAL SIZE: -1.122568093385214


## **3. Majority election**
Now, we instead elect a response based on the levenshtein distance between its response and the majority response. 

In [ ]:
def levenshtein_helper(initial, target):
    if len(target) == 0: return len(initial)
    if len(initial) == 0: return len(target)
    if initial[0] == target[0]: return levenshtein_helper(initial[1:], target[1:])
    return 1 + min(levenshtein_helper(initial[1:], target), levenshtein_helper(initial, target[1:]), levenshtein_helper(initial[1:], target[1:]))

def levenshtein_distance(initial, target):
    initial.sort()
    target.sort()
    return levenshtein_helper(initial=initial, target=target)


distance = decimals.copy()

for i in range(N_JOBS):
    distance[f'sample_{i}'] = distance.apply(lambda row : levenshtein_distance(row[f'sample_{i}'], row['lSolutions']), axis=1)
distance

,sample_0,sample_1,sample_2,sample_3,sample_4,sample_5,sample_6,sample_7,sample_8,sample_9,lSolutions,majority
question_number,,,,,,,,,,,,
0,7,4,7,6,9,6,5,6,5,5,"[9.0, 15.0]","[1.6666666666666667, 3, 5, 9, 15, 24]"
1,5,7,8,5,5,5,5,6,5,5,"[137.0, 388.0]","[4, 388, 776, 137, 525, 2100, 2876]"
2,6,6,6,7,6,6,6,8,9,6,"[4.0, 6.0]","[4, 6, 8.5, 8, 10.5, 10, 85, 93]"
3,6,6,8,9,6,8,6,8,6,8,"[2200.0, 3900.0]","[6, 7, 39, 366, 6100, 405, 2200, 3900]"
4,6,6,7,8,5,6,6,6,6,6,"[10.0, 30.0]","[2, 100, 4, 40, 10, 80, 20, 30]"
...,...,...,...,...,...,...,...,...,...,...,...,...
509,6,6,6,6,7,6,6,6,7,6,"[7.0, 9.0]","[35, -90, 7, 9, 45, 16, 720, 630]"
510,4,4,4,4,4,4,4,4,4,5,[75.0],"[385, 770, 75, 460, 310]"
511,6,6,7,6,6,10,6,6,6,7,"[69.0, 99.0]","[99, 1380, 69, 8, 12, 240, 1620, 30]"


### **Elect representative**
We select the response with the smallest levenshtein distance

In [ ]:
def select_representative(row):
    min_val = float('inf')
    for i in range(N_JOBS):
        min_val = min(row[f'sample_{i}'], min_val)
    
    for i in range(N_JOBS):
        if row[f'sample_{i}'] == min_val: return f'sample_{i}'
    return 'sample_0'

distance['representative'] = distance.apply(lambda row: select_representative(row), axis=1) 
distance.head(1)

,sample_0,sample_1,sample_2,sample_3,sample_4,sample_5,sample_6,sample_7,sample_8,sample_9,lSolutions,majority,representative
question_number,,,,,,,,,,,,,
0,7,4,7,6,9,6,5,6,5,5,"[9.0, 15.0]","[1.6666666666666667, 3, 5, 9, 15, 24]",sample_1


In [ ]:
elected = responses.copy()
elected = elected.join(distance[['representative']])
elected['rep_sample'] = elected.apply(lambda row : row[row['representative']], axis=1)
elected['decimals'] = elected['rep_sample'].apply(lambda row : extract_decimals(row))
elected = elected[['rep_sample', 'decimals', 'lSolutions']]
elected['elected_correct'] = elected.apply(lambda row : check_correct(row['lSolutions'], row['decimals'], base_transform_func), axis=1)
elected

,rep_sample,decimals,lSolutions,elected_correct
question_number,,,,
0,We can use a proportion to solve this problem....,"[3.0, 5, 8, 9, 15, 24]","[9.0, 15.0]",all
1,Let s be the number of student tickets sold an...,"[4, 388, 776, 137, 525, 2100, 2876]","[137.0, 388.0]",all
2,Let x be the number of discs that cost 10.50 d...,"[4, 6, 8.5, 8, 10.5, 10, 85, 93]","[4.0, 6.0]",all
3,"Let x be the amount invested at 7%\nThen, 6100...","[6, 7, 39, 366, 6100, 405, 2200, 3900]","[2200.0, 3900.0]",all
4,Let x be the number of 2 point questions and y...,"[2, 100, 4, 40, 10, 50, 30]","[10.0, 30.0]",all
...,...,...,...,...
509,Let's assume that the dealer sold x etchings a...,"[35, -90, 7, 9, 45, 16, 720, 630]","[7.0, 9.0]",all
510,Let's call the speed of the plane 'p' and the ...,"[385, 770, 75, 460, 310]",[75.0],all
511,Let x be the monthly rental charge for the mic...,"[99, 1380, 69, 8, 12, 240, 1620, 30]","[69.0, 99.0]",all


In [ ]:
value_cnt.join(pandas.DataFrame(elected['elected_correct'].value_counts()))

,sample_0,sample_1,sample_2,sample_3,sample_4,sample_5,sample_6,sample_7,sample_8,sample_9,majority,elected_correct
all,434,435,420,432,425,439,435,423,432,428,444,456
none,68,68,84,69,72,65,71,76,71,70,60,50
some,12,11,10,13,17,10,8,15,11,16,10,8


### **Summary**
I didn't expect the elected one to perform better but I guess it did?

In [ ]:
len_diff = decimals.copy().join(elected[['decimals']])
len_diff['len_difference'] = len_diff.apply(lambda row : len_difference(row, 'decimals'), axis=1)
print('NUMBER OF CASES WHERE AVERAGE INDIVIDUAL SIZE > MAJORITY SIZE:', len(len_diff[len_diff['len_difference'] > 0]))
print('AVERAGE MAJORITY - INDIVIDUAL SIZE:', len_diff['len_difference'].sum()/len(len_diff))

NUMBER OF CASES WHERE AVERAGE INDIVIDUAL SIZE > MAJORITY SIZE: 2
AVERAGE MAJORITY - INDIVIDUAL SIZE: -1.3560311284046693
